<a href="https://colab.research.google.com/github/HanNayeoniee/NLP-study/blob/main/Week%2002-LSTM_char.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### PyTorch LSTM 예제

현재 문장을 주고 다음 문장 예측하기

http://cedartrees.co.kr/index.php/2020/08/01/pytorch-lstm-example/

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
# 'I'가 들어오면 'n'을, 'n'이 들어오면 공백을 예측
sentence = "In the beginning God created the heavens and the earth"

x = sentence[:-1]  # 입력 문장
y = sentence[1:]  # 정답 데이터셋

char_set = list(set(sentence))  # 문장 안의 모든 알파벳
input_size = len(char_set)
hidden_size = len(char_set)

# 각각 문자를 one-hot 형태로 입력하기 위해 만든 딕셔너리
index2char = {i:c for i, c in enumerate(char_set)}
char2index = {c:i for i, c in enumerate(char_set)}

In [3]:
char2index

{' ': 0,
 'G': 15,
 'I': 13,
 'a': 1,
 'b': 5,
 'c': 4,
 'd': 7,
 'e': 12,
 'g': 2,
 'h': 14,
 'i': 10,
 'n': 6,
 'o': 11,
 'r': 16,
 's': 9,
 't': 3,
 'v': 8}

In [6]:
one_hot = []
for i, tkn in enumerate(x):
    one_hot.append(np.eye(len(char_set), dtype='int')[char2index[tkn]])

x_train = torch.Tensor(one_hot)
x_train = x_train.view(1, len(x), -1)

In [7]:
# x_train은 3차원의 형태: [1, 10, 8]
# 1: 문장의 개수, 10: 단어의 개수, 8: 단어의 입력 차원
print(x_train)

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.

In [8]:
y_data = [char2index[c] for c in y]
y_data = torch.Tensor(y_data)

In [9]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        self.rnn = nn.LSTM(
            input_size = input_size,
            hidden_size = hidden_size,
            num_layers = 4,
            batch_first = True,
            bidirectional = True
        )

        self.layers = nn.Sequential(
            nn.ReLU(),
            nn.Linear(input_size*2, hidden_size)
        )

    def forward(self, x):
        y, _ = self.rnn(x)
        y = self.layers(y)
        return y

model = RNN(input_size, hidden_size)
model

RNN(
  (rnn): LSTM(17, 17, num_layers=4, batch_first=True, bidirectional=True)
  (layers): Sequential(
    (0): ReLU()
    (1): Linear(in_features=34, out_features=17, bias=True)
  )
)

In [10]:
# loss & optimizer setting
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())
# start training
for i in range(5000):
    model.train()
    outputs = model(x_train)
    loss = criterion(outputs.view(-1, input_size), y_data.view(-1).long())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i%500 == 0:
        result = outputs.data.numpy().argmax(axis=2)
        result_str = ''.join([char_set[c] for c in np.squeeze(result)])
        print(i, "loss: ", loss.item(), "\nprediction: ", result, "\ntrue Y: ", y_data, "\nprediction str: ", result_str,"\n")

0 loss:  2.8392279148101807 
prediction:  [[7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
  7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 2]] 
true Y:  tensor([ 6.,  0.,  3., 14., 12.,  0.,  5., 12.,  2., 10.,  6.,  6., 10.,  6.,
         2.,  0., 15., 11.,  7.,  0.,  4., 16., 12.,  1.,  3., 12.,  7.,  0.,
         3., 14., 12.,  0., 14., 12.,  1.,  8., 12.,  6.,  9.,  0.,  1.,  6.,
         7.,  0.,  3., 14., 12.,  0., 12.,  1., 16.,  3., 14.]) 
prediction str:  ddddddddddddddddddddddddddddddddddddddddddddddddddddg 

500 loss:  1.782058596611023 
prediction:  [[ 6  0  3  3 12 12 12 12 10 10  6  6  6  6  6  6  0  0  0  0  0 12 12 12
  12 12 12 12 12 12 12 12 12 12 12 12 12 12  0  0  0  0  0  0  0  0 12 12
  12  1 16  3 14]] 
true Y:  tensor([ 6.,  0.,  3., 14., 12.,  0.,  5., 12.,  2., 10.,  6.,  6., 10.,  6.,
         2.,  0., 15., 11.,  7.,  0.,  4., 16., 12.,  1.,  3., 12.,  7.,  0.,
         3., 14., 12.,  0., 14., 12.,  1.,  8., 12.,  6.,  9.,  0.,  1.,  6.,
         7